<a href="https://colab.research.google.com/github/kridtapon/KVO-MACD-Fusion/blob/main/KVO_MACD_Fusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.5/527.5 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.43.0
    Uninstalling llvmlite-0.43.0:
      Successfully uninstalled llvmlite-0.43.0
  Attempting uninstall: numba
    Found existing installation: numba 0.60.0
    Uninstalling numba-0.60.0:
      Successfully unins

In [2]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Klinger Volume Oscillator (KVO)
def klinger_volume_oscillator(df, short_period=34, long_period=55, signal_period=13):
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = np.abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = np.abs(df['Low'] - df['Close'].shift(1))
    df['TrueRange'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    df['VolumeForce'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], -df['Volume'])
    df['KVO'] = (df['VolumeForce'].rolling(window=short_period).sum() -
                 df['VolumeForce'].rolling(window=long_period).sum())
    df['KVO_Signal'] = df['KVO'].rolling(window=signal_period).mean()
    return df['KVO'], df['KVO_Signal']

# Calculate MACD
def calculate_macd(df, fast_period=12, slow_period=26, signal_period=9):
    df['MACD'] = df['Close'].ewm(span=fast_period, min_periods=fast_period).mean() - \
                 df['Close'].ewm(span=slow_period, min_periods=slow_period).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_period, min_periods=signal_period).mean()
    return df['MACD'], df['MACD_Signal']

# Define the stock symbol and time period
symbol = 'SYF'
start_date = '2020-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate KVO and MACD
df['KVO'], df['KVO_Signal'] = klinger_volume_oscillator(df,40,60,5)
df['MACD'], df['MACD_Signal'] = calculate_macd(df,5,20,6)

# Define entry and exit signals based on KVO and MACD
df['Entry'] = (
    (df['KVO'] > df['KVO_Signal']) &  # KVO crosses above KVO Signal
    (df['MACD'] > df['MACD_Signal']) &  # MACD crosses above Signal line
    (df['KVO'].shift(1) <= df['KVO_Signal'].shift(1))  # Previous KVO was below Signal
)

df['Exit'] = (
    (df['KVO'] < df['KVO_Signal']) &  # KVO crosses below KVO Signal
    (df['MACD'] < df['MACD_Signal']) &  # MACD crosses below Signal line
    (df['KVO'].shift(1) >= df['KVO_Signal'].shift(1))  # Previous KVO was above Signal
)

# Convert signals to boolean arrays
entries = df['Entry'].to_numpy()
exits = df['Exit'].to_numpy()

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=entries,
    exits=exits,
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           435072.851647
Total Return [%]                       335.072852
Benchmark Return [%]                    79.508428
Max Gross Exposure [%]                      100.0
Total Fees Paid                      17484.837167
Max Drawdown [%]                        32.681972
Max Drawdown Duration                       422.0
Total Trades                                   37
Total Closed Trades                            37
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            54.054054
Best Trade [%]                          54.167788
Worst Trade [%]                        -15.761454
Avg Winning Trade [%]                   12.241891
Avg Losing Trade [%]                    -4.178278


In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Klinger Volume Oscillator (KVO)
def klinger_volume_oscillator(df, short_period=34, long_period=55, signal_period=13):
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = np.abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = np.abs(df['Low'] - df['Close'].shift(1))
    df['TrueRange'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    df['VolumeForce'] = np.where(df['Close'] > df['Close'].shift(1), df['Volume'], -df['Volume'])
    df['KVO'] = (df['VolumeForce'].rolling(window=short_period).sum() -
                 df['VolumeForce'].rolling(window=long_period).sum())
    df['KVO_Signal'] = df['KVO'].rolling(window=signal_period).mean()
    return df['KVO'], df['KVO_Signal']

# Calculate MACD
def calculate_macd(df, fast_period=12, slow_period=26, signal_period=9):
    df['MACD'] = df['Close'].ewm(span=fast_period, min_periods=fast_period).mean() - \
                 df['Close'].ewm(span=slow_period, min_periods=slow_period).mean()
    df['MACD_Signal'] = df['MACD'].ewm(span=signal_period, min_periods=signal_period).mean()
    return df['MACD'], df['MACD_Signal']

# Define the stock symbol and time period
symbol = 'SYF'
start_date = '2020-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Define parameter grids for KVO and MACD
kvo_params = [(short, long, signal) for short in range(10, 41, 10)
                            for long in range(50, 101, 10)
                            for signal in range(5, 21, 5)]
macd_params = [(fast, slow, signal) for fast in range(5, 21, 5)
                            for slow in range(20, 41, 5)
                            for signal in range(6, 16, 2)]

# Function to calculate strategy returns based on parameters
def run_backtest(short_period, long_period, signal_period, fast_period, slow_period, signal_macd_period):
    # Calculate KVO and MACD
    df['KVO'], df['KVO_Signal'] = klinger_volume_oscillator(df, short_period, long_period, signal_period)
    df['MACD'], df['MACD_Signal'] = calculate_macd(df, fast_period, slow_period, signal_macd_period)

    # Define entry and exit signals based on KVO and MACD
    df['Entry'] = (
        (df['KVO'] > df['KVO_Signal']) &  # KVO crosses above KVO Signal
        (df['MACD'] > df['MACD_Signal']) &  # MACD crosses above Signal line
        (df['KVO'].shift(1) <= df['KVO_Signal'].shift(1))  # Previous KVO was below Signal
    )

    df['Exit'] = (
        (df['KVO'] < df['KVO_Signal']) &  # KVO crosses below KVO Signal
        (df['MACD'] < df['MACD_Signal']) &  # MACD crosses below Signal line
        (df['KVO'].shift(1) >= df['KVO_Signal'].shift(1))  # Previous KVO was above Signal
    )

    # Convert signals to boolean arrays
    entries = df['Entry'].to_numpy()
    exits = df['Exit'].to_numpy()

    # Backtest using vectorbt
    portfolio = vbt.Portfolio.from_signals(
        close=df['Close'],
        entries=entries,
        exits=exits,
        init_cash=100_000,
        fees=0.001
    )

    return portfolio

# Optimize parameters using vectorbt's vectorized backtest
best_total_return = -np.inf
best_params = None
best_portfolio = None

# Grid search for the best parameters
for kvo_param in kvo_params:
    for macd_param in macd_params:
        short_period, long_period, signal_period = kvo_param
        fast_period, slow_period, signal_macd_period = macd_param

        # Run backtest for the given parameter combination
        portfolio = run_backtest(short_period, long_period, signal_period, fast_period, slow_period, signal_macd_period)

        # Calculate performance metrics (Total Return for optimization)
        total_return = portfolio.stats()['Total Return [%]']

        # Check if the current combination yields the best Total Return
        if total_return > best_total_return:
            best_total_return = total_return
            best_params = (kvo_param, macd_param)
            best_portfolio = portfolio

# Display the best performance
print(f"Best Parameters for KVO: {best_params[0]}")
print(f"Best Parameters for MACD: {best_params[1]}")
print("Best Portfolio Performance (Total Return):", best_total_return)

# Plot the best portfolio's performance
best_portfolio.plot().show()


Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'calmar_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'omega_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sortino_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning: Metric 'sharpe_ratio' requires frequency to be set
  warnings.warn(warning_message)
/usr/local/lib/python3.10/dist-packages/vectorbt/generic/stats_builde

Best Parameters for KVO: (40, 60, 5)
Best Parameters for MACD: (5, 20, 6)
Best Portfolio Performance (Total Return): 335.07285164671237
